## Загрузка библиотек

In [0]:
!pip install -qq transformers
!pip install watermark

In [0]:
import watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers,sklearn.linear_model, plotly

In [0]:
import transformers
import torch
import io
import time

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from matplotlib import rc
from pylab import rcParams

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
# from collections import defaultdict
# from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pd.set_option('display.max_colwidth', 200)


Настройки отображения графиков

In [0]:
# !pip install cufflinks --upgrade
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js?x49997"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## Подготовка обучающей выборки

Загружаем размеченную выборку

In [0]:
!gdown --id 1MSfU7VGOlXSGdObF5hSE7XDnZ5frz0zc

Смотрим на данные:

In [0]:
df = pd.read_csv('Training_data.csv', 
                  header=0)

In [0]:
df = df.drop_duplicates(subset=['Sentence'])

In [0]:
df = df[:1700]

In [0]:
df['word_count'] = df.Sentence.apply(lambda x: len(x.split()))

In [0]:
df.head()

In [0]:
df.shape

In [0]:
df.Score.unique()

In [0]:
max(df.word_count)

## EDA

Проводим визуальный анализ:

In [0]:
class_names = ['Нейтральная', 'Положительная', 'Отрицательная']

In [0]:
ax = sns.countplot(df.Score, order =df.Score.value_counts().index)
colors = ["#87CEFA", "#90EE90", "#FA8072"]
sns.set_palette(sns.color_palette(colors))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)
plt.xlabel('Тональность отчетов');
plt.ylabel('Количество');
ax.set_xticklabels(class_names);

In [0]:
configure_plotly_browser_state()

df[df['Score']==1]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='red',
    yTitle='count',
    title='Positive Text Length Distribution')

df[df['Score']==-1]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='green',
    yTitle='count',
    title='Negative Text Length Distribution')

df[df['Score']==0]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    yTitle='count',
    title='Neutral Text Length Distribution')

## Обучение BERT

### Токенизация

In [0]:
from transformers import AutoTokenizer

In [0]:
tkz = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
sample_txt = 'Развитие компании проходит в условиях повышенной конкуренции'

In [0]:
tokens_tkz = tkz.tokenize(sample_txt)
tokens_tkz

In [0]:
text_encoded = tkz.encode(tokens_tkz)
text_encoded

In [0]:
text_decode = tkz.decode(text_encoded)
text_decode 

### Загрузка модели

In [0]:
from transformers import AutoModelWithLMHead

In [0]:
model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
# model = AutoModelWithLMHead.from_pretrained("distilbert-base-multilingual-cased") # 0.5733

### Подготовка данных

In [0]:
df.Sentence = df.Sentence.map(lambda x: str(x).lower()) # переводим символы в нижний регистр

In [0]:
df['tokenized'] = df.Sentence.apply((lambda x: tkz.encode(tkz.tokenize(x))))

In [0]:
df.head()

Так как обучение BERT требует много памяти, разобьем обучающую выборку на несколько частей и будем последовательно обучать модель

In [0]:
df_1 = df[:200]
df_2 = df[200:400]
df_3 = df[400:600]
df_4 = df[600:800]
df_5 = df[800:1000]
df_6 = df[1000:1100]
df_7 = df[1100:1200]
df_8 = df[1200:1300]
df_9 = df[1300:1400]
df_10 = df[1400:1500]
df_11 = df[1500:1600]
df_12 = df[1600:]
df_list = df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12

In [0]:
len(df_list)

In [0]:
padded = np.array([i + [0]*(max_len-len(i)) for i in df_1.tokenized.values])

In [0]:
np.array(padded).shape

In [0]:
np.array(padded)[2]

Masking

In [0]:
# attention_mask = np.where(padded != 0, 1, 0)
# attention_mask.shape

### Обучение модели

Собираем обработанные данные

In [0]:
def train_model(df, features_list):

  # Токенизируем предложения
  df['tokenized'] = df.Sentence.apply((lambda x: tkz.encode(tkz.tokenize(x))))

  # Вычисляем длину наибольшего токена
  max_len = 0
  for index, i in df.tokenized.iteritems():
      if len(i) > max_len:
          max_len = len(i)

  # Создаем массив и заполняем его токенами
  padded = np.array([i + [0]*(max_len-len(i)) for i in df.tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  # Обучаем модель

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)  

  # Сохраняем признаки
  features = last_hidden_states[0][:,0,:].numpy()
  features_list.append(features)

  return None

In [0]:
features_list_3 = []

In [0]:
for i in df_list[8:]:
  prepare_df(i, features_list=features_list_3)
  time.sleep(5)

In [0]:
features_list_3[:2]

In [0]:
features_3 = np.vstack(features_list_3)
features_3

In [0]:
filename = '/content/gdrive/My Drive/bert_features_3'
np.save(file=filename,arr=features_3)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp "/content/gdrive/My Drive/bert_features_1.npy" "bert_features_1.npy"
!cp "/content/gdrive/My Drive/bert_features_2.npy" "bert_features_2.npy"
!cp "/content/gdrive/My Drive/bert_features_3.npy" "bert_features_3.npy"

In [0]:
feat_1 = np.load('bert_features_1.npy')
feat_2 = np.load('bert_features_2.npy')
feat_3 = np.load('bert_features_3.npy')
print(feat_1.shape)
print(feat_2.shape)
print(feat_3.shape)

In [0]:
features = np.vstack([feat_1,feat_2,feat_3])
features.shape

In [0]:
labels = df.Score

In [0]:
labels.value_counts()

### Train-test Split

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25)

## Выбор классификатора

#### Dummy Classifier

In [0]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent')

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#### Логистическая регрессия

##### Тренируем модель

In [0]:
lr_clf = LogisticRegression(max_iter=1000, C=4.0)
lr_clf.fit(train_features, train_labels)

In [0]:
accuracy = lr_clf.score(test_features, test_labels)
print('LR Accuracy:', accuracy)

In [0]:
from sklearn.metrics import classification_report

In [0]:
labels_test = test_labels.to_list()

In [0]:
labels_pred = lr_clf.predict(test_features).tolist()

In [0]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

##### Сохраняем модель

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pickle
filename = 'lr_model.sav'
pickle.dump(lr_clf, open(filename, 'wb'))

#### SVM

In [0]:
from sklearn import svm

##### Тренируем модель

In [0]:
svm_clf = svm.SVC(kernel='linear', C=4.0) # Linear Kernel

In [0]:
svm_clf.fit(train_features, train_labels)

In [0]:
svm_clf.fit(train_features, train_labels)

In [0]:
labels_test = test_labels.to_list()

In [0]:
accuracy = svm_clf.score(test_features, test_labels)
print('SVM Accuracy:', accuracy)

In [0]:
labels_pred = svm_clf.predict(test_features).tolist()

In [0]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB

##### Тренируем модель

In [0]:
gnb_clf = GaussianNB()

In [0]:
gnb_clf.fit(train_features, train_labels)

In [0]:
accuracy = gnb_clf.score(test_features, test_labels)
print('GNB Accuracy:', accuracy)

In [0]:
labels_pred = gnb_clf.predict(test_features).tolist()

In [0]:
labels_test = test_labels.to_list()

In [0]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Decision Trees

In [0]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

##### Тренируем модель

In [0]:
dtc_clf = DecisionTreeClassifier()

In [0]:
dtc_clf.fit(train_features, train_labels)

In [0]:
accuracy = dtc_clf.score(test_features, test_labels)
print('Decision Tree Accuracy:', accuracy)

In [0]:
labels_pred = dtc_clf.predict(test_features).tolist()

In [0]:
labels_test = test_labels.to_list()

In [0]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

##### Тренируем модель

In [0]:
rf_clf=RandomForestClassifier(n_estimators=100)

In [0]:
rf_clf.fit(train_features, train_labels)

In [0]:
accuracy = rf_clf.score(test_features, test_labels)
print('Decision Tree Accuracy:', accuracy)

Наилучшей моделью оказался **случайный лес**, показавший точность 0.83 на тестовой выборке.

In [0]:
labels_pred = rf_clf.predict(test_features).tolist()

In [0]:
labels_test = test_labels.to_list()

In [0]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

In [0]:
df.shape

In [0]:
rf_clf.fit(train_features, train_labels)

##### Сохраняем модель

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pickle
filename = 'rf_model.sav'
pickle.dump(rf_clf, open(filename, 'wb'))

##### Загружаем модель

In [0]:
!cp "/content/gdrive/My Drive/rf_model.sav" 'rf_model.sav'

In [0]:
xtimport pickle
filename = 'rf_model.sav'
rf_clf = pickle.load(open(filename, 'rb'))
rf_clf
# result = lr_clf.score(test_features, test_labels)
# print(result)

### Матрица ошибок

In [0]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('Правильная тональность')
  plt.xlabel('Предсказанная тональность');

In [0]:
cm = confusion_matrix(labels_test, labels_pred)
cm

In [0]:
df_pred = pd.DataFrame(data=list(zip(labels_test,labels_pred)), columns=['True','Predicted'])
df_pred

In [0]:
class_names = ['Негативная', 'Нейтральная', 'Положительная']

In [0]:
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

## Генерируем предсказания 

#### По тексту

In [0]:
txt = [
       'Компания нацелена на предупреждение, сокращение и минимизацию последствий разливов нефти и нефтепродуктов',
       'В Компании сформирована и развивается система оперативного реагирования на разливы нефти и нефтепродуктов, их локализации и ликвидации с целью минимизации экологических последствий, в том числе влияния на водные ресурсы',
       ' В части «Раскрытия информации» место Компании в рейтинге экологической ответственности поднялось на восемь пунктов, с десятого на второе место по сравнению с предыдущим периодом',
       'Главный актив Компании – это высокопрофессиональный персонал, мотивированный на эффективную работу',
       'Повышение эффективности труда остается одним из ключевых приоритетов Компании. В рамках реализации этой задачи в 2018 году актуализированы внутрикорпоративные методики расчета показателей производительности труда по Компании в целом, по основным бизнес-блокам и Обществам Группы основных бизнес-блоков',
       'В Компании разработан перечень мероприятий по росту производительности труда в Компании. Мероприятия включены в Долгосрочную программу развития Компании, отчет по исполнению которой происходит на ежегодной основе',
       'Компания подвержена влиянию множества присущих нефтегазовой отрасли рисков, основными из которых являются снижение цен на нефть и нефтепродукты и рост цен на приобретаемое сырье и услуги',
       'Компания ведет мониторинг законопроектов, что позволяет заблаговременно оценить последствия предлагаемых изменений и учесть их в своих планах. Компания имеет обширный опыт реализации проектов в области добычи и переработки углеводородного сырья, а также обладает финансовыми, материально-техническими и кадровыми ресурсами, необходимыми для выполнения обязательств по лицензионным соглашениям',
       'Характерными для Компании финансовыми рисками являются валютный, процентный, инфляционный, кредитный риски и риск ликвидности. Данные риски могут негативно повлиять на финансовые результаты деятельности Компании вследствие роста расходов, обесценения активов, снижения рентабельности и денежного потока Компании',
       'В региональном конкурсе реализованных проектов в области энергосбережения и повышения энергоэффективности ENES–2018 Компания признана победителем в двух номинациях: «Лучшая реализованная комплексная программа в ТЭК по популяризации энергосбережения и повышения энергоэффективности» и «Эффективная система управления в области энергосбережения и повышения энергоэффективности на предприятиях ТЭК»',
       'Важное значение Компания придает программам, направленным на развитие и поддержку массового спорта, физического развития сотрудников и их детей',
       'В целях повышения качества комплектования квалифицированными кадрами требуемых профессий и нужной квалификации в Компании проведено 7539 тестирований кандидатов на трудоустройство по 100 рабочим профессиям на базе компьютерных классов',
       'Компания нацелена на предупреждение, сокращение и минимизацию последствий разливов нефти',
       'В блоке «Нефтепереработка и нефтегазохимия» реализуется комплекс программ по обеспечению целостности оборудования и исключению аварийных ситуаций с неблагоприятными экологическими последствиями',
       'Компания является одним из крупнейших работодателей в России'
]

In [0]:
txt = [input()]

In [0]:
txt_tk = [tkz.encode(tkz.tokenize(x)) for x in txt]

In [0]:
max_len = 0
for i in txt_tk:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

In [0]:
padded = np.array([i + [0]*(max_len-len(i)) for i in txt_tk])

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
features

In [0]:
preds = list(rf_clf.predict(features))
preds

In [0]:
classes_dict = {-1:'negative', 0:'neutral', 1:'positive'}
classes_dict

In [0]:
preds_ls = list(zip(txt, preds))

In [0]:
for k,v in preds_ls:
  print(classes_dict.get(v), k, sep=' : ')

#### По обучающей выборке

In [0]:
df_test = df[15:215]
df_test.head()

In [0]:
max_len = 0
for i in df_test.tokenized:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

In [0]:
padded = np.array([i + [0]*(max_len-len(i)) for i in df_test.tokenized])

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
preds = list(rf_clf.predict(features))

In [0]:
classes_dict = {-1:'negative', 0:'neutral', 1:'positive'}
classes_dict

In [0]:
preds_ls = list(zip(df_test.Score, preds))

In [0]:
df_preds_test = pd.DataFrame(preds_ls, columns=['True','Predicted'])
df_test['Preds']=preds
df_test[90:120]

In [0]:
df_test[df_test.Score != df_test.Preds][:10]

#### Для отчетов

In [0]:
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [0]:
tkz = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
def prepare_df(df):
  df = df.drop_duplicates(subset=['Sentence'])
  df.Sentence = df.Sentence.map(lambda x: str(x).lower()) # переводим символы в нижний регистр
  df['tokenized'] = df.Sentence.apply(lambda x: tkz.encode(tkz.tokenize(x)))
  display(df.head())
  print(df.shape)
  return df

In [0]:
def get_sentiment(df, max_length=50):
  
  max_len = 0
  long_ls = []
  for index, i in df.tokenized.iteritems():
    if len(i) > max_len:
        max_len = len(i)
    if len(i) > max_length:
      long_ls.append(index)

  print('Максимальная длина токена:', max_len)
  print('Предложения, превышающие ограничение по длине:', len(long_ls))

  df = df.drop(long_ls)
  print(df.shape)

  padded = np.array([i + [0]*(max_length-len(i)) for i in df.tokenized.values])

  attention_mask = np.where(padded != 0, 1, 0)
  print(attention_mask.shape)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  print('-'*20)
  print('Обучение модели')
  print('-'*20) 
  begin = time.time()

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  
  end = time.time()
  print('Обучение завершено')
  print('Обучение заняло:', "{:.3f}".format(round(end-begin, 3)), 'секунд')
  print('-'*20)

  features = last_hidden_states[0][:,0,:].numpy()

  # predicted = list(rf_clf.predict(features))
  predicted = list(lr_clf.predict(features))
  pred_dict = dict()

  for i in set(predicted):
    if i not in pred_dict.keys():
      pred_dict[i] = predicted.count(i)
  
  df['preds'] = predicted
  preds = pd.DataFrame(data=predicted, columns=['Prediction'])

  assert len(df) == len(preds)

  ton = np.mean(predicted)
  neg = pred_dict.get(-1,0) / len(predicted)
  pos = pred_dict.get(1,0) / len(predicted)
  neut = pred_dict.get(0,0) / len(predicted)
  stats = dict(neutral=neut, positive=pos, negative=neg, tonality=ton)
  df_stats = pd.DataFrame(data=stats, index=[0])

  return df, df_stats

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# from google.colab import files

# uploaded = files.upload()

Меняем название файла на текущий

In [0]:
!ls "/content/gdrive/My Drive/"

In [0]:
!cp "/content/gdrive/My Drive/NVTK_17.csv" "NVTK_17.csv"

In [0]:
df = pd.read_csv('NVTK_17.csv', header=0,
                  sep=',')

In [0]:
# df.head(2)
# df.drop(columns=['Sector','Score'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [0]:
df.head(2)

In [0]:
df = prepare_df(df)

In [0]:
df_1 = df[1000:]
# df_1 =df

In [0]:
df_final, df_stats = get_sentiment(df_1, max_length=62)

In [0]:
df_stats

In [0]:
neut = 500*(0.716+0.724) + 209*0.770335
pos = 500*(0.262+0.2)+ 209*0.205742
neg = 500*(0.022+0.076)+ 209*0.023923
total = neut+neg+pos
ton = (pos-neg)/total

In [0]:
stats = dict(neutral=neut/total, positive=pos/total, negative=neg/total, tonality=ton)
df_stats = pd.DataFrame(data=stats, index=[0])
df_stats

In [0]:
df_final[df_final.preds==-1][:8]